In [18]:
%pip install langchain_community langchain_ollama transformers docling --quiet 

ERROR! Session/line number was not unique in database. History logging moved to new session 5
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [48]:
from langchain_ollama.llms import OllamaLLM
from transformers import AutoTokenizer

import itertools
from typing import Iterator, Callable
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
import pandas as pd

In [22]:
data = "D:\Code\local-agent-notebooks\datasets\cafef-news-09-07-2025.csv"
df = pd.read_csv(data)
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Admin\AppData\Local\Temp\ipykernel_1540\1940478403.py:1: SyntaxWarning: invalid escape sequence '\C'
  data = "D:\Code\local-agent-notebooks\datasets\cafef-news-09-07-2025.csv"


,Unnamed: 0,url,title,published,summary,body
0,59,https://cafef.vn/dn-lien-quan-truong-my-lan-na...,DN liên quan Trương Mỹ Lan nắm trong tay loạt ...,2025-09-07 11:51:00,Đơn vị kiểm toán (Công ty A&C) đã từ chối đưa ...,MỚI NHẤT!\n\n07-09-2025 - 11:51 AM | Doanh ngh...
1,60,https://cafef.vn/tin-vui-cho-evn-1882509071135...,Tin vui cho EVN,2025-09-07 11:35:00,Lần gần nhất có thay đổi giá bán lẻ điện bình ...,MỚI NHẤT!\n\n07-09-2025 - 11:35 AM | Doanh ngh...
2,11,https://cafef.vn/gia-nha-tang-cao-chung-cu-so-...,"Giá nhà tăng cao, chung cư sở hữu có thời hạn ...",2025-09-07 11:23:00,"Thời gian gần đây, giá nhà tăng quá cao khiến ...",MỚI NHẤT!\n\n07-09-2025 - 11:23 AM | Bất động ...
3,70,https://cafef.vn/giam-doc-proship-nam-2015-gia...,Giám đốc Proship: Năm 2015 giao hàng nội thành...,2025-09-07 11:23:00,Vị này lý giải chi phí giao hàng hiện đã rẻ hơ...,MỚI NHẤT!\n\n07-09-2025 - 11:23 AM | Doanh ngh...
4,12,https://cafef.vn/can-canh-khu-nha-xuong-700m2-...,Cận cảnh khu nhà xưởng 700m2 cháy rụi sau vài ...,2025-09-07 11:22:00,"Khu nhà xưởng ở phường Việt Hưng, TP Hà Nội có...",MỚI NHẤT!\n\n07-09-2025 - 11:22 AM | Bất động ...


In [25]:
example = texts = df['body'][1]
example

'MỚI NHẤT!\n\n07-09-2025 - 11:35 AM | Doanh nghiệp\n\nBộ Công Thương vừa có tờ trình về dự thảo sửa đổi bổ sung Nghị định 72/2025 về cơ chế, thời gian điều chỉnh giá bán lẻ điện bình quân. Đây là dự thảo liên quan đến việc cho phép Tập đoàn Điện lực Việt Nam (EVN) thu hồi khoản lỗ của các năm trước thông qua việc điều chỉnh giá bán lẻ điện trong các năm tiếp theo.\n\nCụ thể, EVN cho biết lỗ lũy kế năm 2022-2023 khoảng 50.029 tỷ đồng. Còn riêng công ty mẹ EVN, lỗ lũy kế đến hết năm 2024 khoảng 44.792 tỷ đồng.\ufeff\n\nBộ Công thương đưa ra 2 phương án để xin ý kiến về việc thu hồi khoản lỗ.\n\nPhương án 1: Cho phép phân bổ các khoản lỗ phát sinh từ năm 2022 trở đi vào giá điện của các năm sau. Đây là một cơ chế áp dụng lâu dài.\n\nPhương án 2: Chỉ cho phép xử lý các khoản lỗ phát sinh từ năm 2022 đến trước khi Nghị định sửa đổi có hiệu lực. Phương án này chỉ nhằm xử lý vấn đề đã tồn tại.\n\nVề tác động của sửa đổi nghị định, Bộ\xa0Công Thương cho biết tình hình thủy văn 7 tháng đầu năm 

In [57]:
llm = OllamaLLM(model="granite3.3:8b", temperature=0)

In [84]:
prompt_template = """Write a concise summary of the following news article.:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [85]:
docs = [Document(page_content=example)]

In [86]:
# Summarization chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

In [87]:
result = stuff_chain.invoke(docs)

2025-09-07 15:39:41,446 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


In [88]:
result.keys()

dict_keys(['input_documents', 'output_text'])

In [89]:
print(result['output_text'])

The Vietnamese Ministry of Finance has proposed amendments to Decree 72/2025 concerning the mechanism and timeline for adjusting retail electricity prices. This proposal aims to allow State-owned Vietnam Electricity Corporation (EVN) to recover previous year's losses through price adjustments for retail electricity in subsequent years. EVN reported cumulative losses of approximately 50.029 trillion VND from 2022-2023 and 44.792 trillion VND for its parent company by the end of 2024.

Two options have been presented:
1. Distribute past losses over future retail electricity price adjustments, a long-term application method.
2. Address existing past losses from 2022 until the decree amendment takes effect, focusing on resolving current issues without broad implications.

The Ministry states that favorable economic conditions in the first seven months of 2025 have positively impacted EVN's production and business outcomes, minimizing the impact of these proposed changes on retail electrici

In [90]:
# Communication Services/Dịch vụ viễn thông
# Consumer Discretionary/Hàng tiêu dùng
# Consumer Staples/Hàng tiêu dùng thiết yếu
# Health Care/Chăm sóc sức khỏe
# Financial/Tài chính
# Materials/Nguyên vật liệu
# Utilities/Dịch vụ tiện ích
# Energy/Năng lượng
# Industrials/Công nghiệp
# Information Technology/Công nghệ thông tin
# Real Estate/Bất động sản

prompt = PromptTemplate(
    input_variables=["news"],
    template="""
    Given this news:

    {news}

    Which stock market sectors are affected? 
    Choose only from: utilities, industrials, consumer discretionary, consumer staples, financials,
    energy, real estate, IT, materials, health care, communication.
    Answer as a list of sector names.
    """
)

chain = LLMChain(llm=OllamaLLM(model="granite3.3:8b"), prompt=prompt)

news = result['output_text']
sectors = chain.run(news)

2025-09-07 15:40:57,470 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


In [91]:
print(sectors)

- Utilities

The primary stock market sector affected by this news is utilities, as the proposed amendments to Decree 72/2025 directly concern retail electricity prices in Vietnam. The adjustments aim to enable State-owned Vietnam Electricity Corporation (EVN) to recover previous years' losses through price changes for retail electricity. As a result, utility companies and their stocks could be influenced by these amendments and resulting price adjustments, potentially impacting investor decisions in this sector. Other sectors like industrials might also indirectly experience some ripple effects due to increased energy costs affecting manufacturing processes and operations; however, utilities is the most directly affected sector mentioned in the given news.
